In [5]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import dash
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_daq as daq

In [6]:
df = pd.read_csv('../data/weatherAUS.csv')
climate = pd.read_csv('../data/Location_Climate_unique.csv').set_index('Location')['Climate'].to_dict()

# print(df.isna().sum() / len(df))
# print(df.dtypes)
# climate

print(df.dtypes)

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object


In [9]:
df.duplicated().sum()

0

In [21]:
df_all = df.copy()

In [4]:
fig_all = px.imshow(df_all.corr(), text_auto = True, aspect = 'auto', title = 'Heatmap Basique', color_continuous_scale='RdBu', height = 1000)
fig_all.show()

NameError: name 'px' is not defined

In [22]:
df_all_raintomorrow = df_all.copy()

df_all_raintomorrow['RainTomorrow'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)

col = df_all_raintomorrow.pop('RainTomorrow')
df_all_raintomorrow.insert(0, 'RainTomorrow', col)

fig_all_raintomorrow = px.imshow(df_all_raintomorrow.corr().round(2)['RainTomorrow'].to_frame(), text_auto = True, aspect = 'square',
                                 color_continuous_scale='RdBu', title = 'Heatmap Basique avec RainTomorrow (0/1)',
                                 width = 500)
fig_all_raintomorrow.show()

# df_lau.corr().round(2)['RainTomorrow'].to_frame()

In [192]:
df_all_include = df_all.copy()

# on garde la valeur du mois pour insérer la notion de date dans le heatmap
# df_all_include['Date'] = pd.to_datetime(df_all_include['Date'])
# df_all_include['Months'] = df_all_include['Date'].dt.month

# #classification des ville par climate pour supprimer la colonne ville et remplacer climat par label
# df_all_include['climate'] = df_all_include['Location'].map(climate)

# le = LabelEncoder()
# df_all_include['climate'] = le.fit_transform(df_all_include['climate'])


#label encoding des vents (toutes les coloones vent ont les memes caractéristique en nombre de valeur)
le2 = LabelEncoder()
df_all_include['WindGustDir'] = le2.fit_transform(df_all_include['WindGustDir'])
df_all_include['WindDir9am'] = le2.transform(df_all_include['WindDir9am'])
df_all_include['WindDir3pm'] = le2.transform(df_all_include['WindDir3pm'])

#remplacement des valeurs de RainToday et RainTomorrow par 0 et 1
df_all_include['RainTomorrow'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)
df_all_include['RainToday'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)

col = df_all_include.pop('RainTomorrow')
df_all_include.insert(0, 'RainTomorrow', col)

# df_all_include.drop(columns = ['Date', 'Location'], inplace = True)

df_all_include.head()


,RainTomorrow,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,0.0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,13,44.0,...,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0.0
1,0.0,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,14,44.0,...,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0.0
2,0.0,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,15,46.0,...,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0.0
3,0.0,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,4,24.0,...,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0.0
4,0.0,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,13,41.0,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0


In [195]:
df_all_include.corr().round(2)['RainTomorrow']

RainTomorrow     1.00
MinTemp          0.08
MaxTemp         -0.16
Rainfall         0.24
Evaporation     -0.12
Sunshine        -0.45
WindGustDir      0.05
WindGustSpeed    0.23
WindDir9am       0.02
WindDir3pm       0.03
WindSpeed9am     0.09
WindSpeed3pm     0.09
Humidity9am      0.26
Humidity3pm      0.45
Pressure9am     -0.25
Pressure3pm     -0.23
Cloud9am         0.32
Cloud3pm         0.38
Temp9am         -0.03
Temp3pm         -0.19
RainToday        0.31
Name: RainTomorrow, dtype: float64

In [193]:
fig_all_include = px.imshow(df_all_include.corr().round(2), text_auto = True, aspect = 'auto', color_continuous_scale='RdBu',
                    title = 'Heatmap avec toutes valeurs qualitatives encodées (Date --- > months, Location --> climate) et mesure de vent',
                    width = 1000,
                    height = 600)
fig_all_include.show()

In [110]:
#on drop les colonnes qui on plus de 30% de valeurs manquantes

df_drop_col = df_all_include.drop(columns = ['Sunshine', 'Evaporation', 'Cloud9am', 'Cloud3pm'])


In [168]:
fig_drop_col = px.imshow(df_drop_col.corr().round(2)['RainTomorrow'].to_frame(), text_auto = True, aspect = 'auto', color_continuous_scale='RdBu',
                         title = 'Heatmap tout encoder avec drop des colonnes : Sunshine, Evaporation, Cloud9am, Cloud3pm',
                         width = 500,
                         height = 600)
fig_drop_col.show()

In [112]:
#je drop tout les na sans disctinctions

fig_drop_col_na = px.imshow(df_drop_col.dropna().corr(), text_auto = True, aspect = 'auto', color_continuous_scale='RdBu',
                            title = 'Heatmap, tout encoder, drop col, et dropna sauvage')
fig_drop_col_na.show()

In [109]:
#je recommence en utilisant le onehotencoder


df_all_include_dum = df_all.copy()

# on garde la valeur du mois pour insérer la notion de date dans le heatmap
df_all_include_dum['Date'] = pd.to_datetime(df_all_include_dum['Date'])
df_all_include_dum['Months'] = df_all_include_dum['Date'].dt.month

#classification des ville par climate pour supprimer la colonne ville et remplacer climat par label
df_all_include_dum['climate'] = df_all_include_dum['Location'].map(climate)


df_all_include_dum = pd.get_dummies(df_all_include_dum, columns=['climate'])
df_all_include_dum = pd.get_dummies(df_all_include_dum, columns=['WindGustDir'])
df_all_include_dum = pd.get_dummies(df_all_include_dum, columns=['WindDir9am'])
df_all_include_dum = pd.get_dummies(df_all_include_dum, columns=['WindDir3pm'])

#remplacement des valeurs de RainToday et RainTomorrow par 0 et 1
df_all_include_dum['RainTomorrow'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)
df_all_include_dum['RainToday'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)



df_all_include_dum.drop(columns = ['Date', 'Location'], inplace = True)

df_all_include_dum.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,...,0,0,0,0,0,0,0,0,1,0
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,...,0,0,0,0,0,0,0,0,0,1
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,...,0,0,0,0,0,0,0,0,0,1
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,...,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,...,0,1,0,0,0,0,0,0,0,0


In [184]:
df_syd = df[df['Location'] == 'Sydney']

df_syd['RainTomorrow'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)
df_syd['RainToday'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)

# print(df_syd.isna().sum()/len(df_syd))


# on garde la valeur du mois pour insérer la notion de date dans le heatmap
df_syd['Date'] = pd.to_datetime(df_syd['Date'])
df_syd['Months'] = df_syd['Date'].dt.month


col = df_syd.pop('RainTomorrow')
df_syd.insert(0, 'RainTomorrow', col)



df_wol = df[df['Location'] == 'Wollongong']

df_wol['RainTomorrow'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)
df_wol['RainToday'].replace(to_replace = ['No', 'Yes'], value = [0, 1], inplace = True)

# print(df_wol.isna().sum()/len(df_wol))


# on garde la valeur du mois pour insérer la notion de date dans le heatmap
df_wol['Date'] = pd.to_datetime(df_wol['Date'])
df_wol['Months'] = df_wol['Date'].dt.month


col = df_wol.pop('RainTomorrow')
df_wol.insert(0, 'RainTomorrow', col)

/tmp/ipykernel_183439/1624606516.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_183439/1624606516.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_183439/1624606516.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_183439/1624606516.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a Data

In [171]:
fig_syd = px.imshow(df_syd.corr().round(2)['RainTomorrow'].to_frame(), text_auto = True, aspect = 'auto',
                    color_continuous_scale='RdBu', width = 500, title = 'Sydney')
fig_syd.show()

In [189]:
# matrix_lau = df_lau.corr().round(2)
# matrix_lau

fig_wol = px.imshow(df_wol.drop(columns = ['Sunshine', 'Evaporation']).corr().round(2)['RainTomorrow'].to_frame(), title = "Wollongong",
                    text_auto = True, aspect = 'auto', color_continuous_scale='RdBu', width = 500)
fig_wol.show()

In [160]:
# fig_all
# fig_all_raintomorrow
# fig_all_include
# fig_drop_col
# fig_drop_col_na
# fig_syd
# fig_lau


fig = go.Figure()

# Ajouter les traces
fig.add_trace(fig_all.data[0])
fig.add_trace(fig_all_raintomorrow.data[0])
fig.add_trace(fig_all_include.data[0])
fig.add_trace(fig_drop_col.data[0])
fig.add_trace(fig_drop_col_na.data[0])
fig.add_trace(fig_syd.data[0])
fig.add_trace(fig_lau.data[0])

fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'label': '1 - Heatmap basique',
                    'method': 'update',
                    'args': [{'visible': [True, False, False, False, False, False, False, False]},
                             {'title': '1 - Matrice de Corrélation (Toutes les données)'}]
                },
                {
                    'label': '2 - Matrice avec Tomorrow',
                    'method': 'update',
                    'args': [{'visible': [False, True, False, False, False, False, False, False]},
                             {'title': '2 - Matrice de Corrélation (Toutes les données) + Tomorrow (0/1)'}]
                },
                {
                    'label': '3 - Matrice encoder',
                    'method': 'update',
                    'args': [{'visible': [False, False, True, False, False, False, False, False]},
                             {'title': '3 - Matrice de Corrélation + quantitative encoder'}]
                },
                # Ajouter d'autres boutons pour les autres graphiques
                {
                    'label': '4 - Matrice Drop Col',
                    'method': 'update',
                    'args': [{'visible': [False, False, False, True, False, False, False, False]},
                             {'title': '4 - Matrice de Corrélation après suppression des colonnes Sunshine, Evaporation, Cloud9am, Cloud3pm'}]
                },
                {
                    'label': '5 - Matrice Drop Col Na',
                    'method': 'update',
                    'args': [{'visible': [False, False, False, False, True, False, False, False]},
                             {'title': '5 - Matrice de Corrélation avec Na supprimés'}]
                },
                {
                    'label': '6 - Matrice Sydney',
                    'method': 'update',
                    'args': [{'visible': [False, False, False, False, False, True, False, False]},
                             {'title': '6 - Matrice de Corrélation pour Sydney sans dropcol, sans dropna'}]
                },
                {
                    'label': '6 - Matrice Launceston',
                    'method': 'update',
                    'args': [{'visible': [False, False, False, False, False, False, True]},
                             {'title': '6 - Matrice de Corrélation pour Launceston sans dropcol, sans dropna mais beacoup de Sunshine manque'}]
                },

            ],
            'direction': 'down',
            'showactive': True,
            'active': 0,
            'x': 0.8,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top'
        }
    ],
    showlegend=True,
    width=1400,
    height=600,
)

fig.data[0].update(visible=True)
fig.data[1].update(visible=False)
fig.data[2].update(visible=False)
fig.data[3].update(visible=False)
fig.data[4].update(visible=False)
fig.data[5].update(visible=False)
fig.data[6].update(visible=False)

fig.show()

In [23]:
a = df_all_raintomorrow.corr(method = 'spearman')


fig_cat = px.imshow(a, title = 'Spearman categorielle',
                    text_auto = True, aspect = 'auto', color_continuous_scale='RdBu', width = 1000)
fig_cat.show()